In [ ]:
#Using Spacy

In [1]:
# !pip install spacy
# !python -m spacy download en_core_web_sm

import spacy

# Load the spaCy model
nlp = spacy.load("en_core_web_sm")


In [3]:
# Acronym mapping
acronym_map = {
    "CD": "Closing Date",
    "IP": "Investment Period",
    "DI": "Direct Investment",
    "AFF": "Affiliate",
    "GP": "General Partner",
    "LI": "Lead Investor",
    "SIF": "Secondary Investee Fund",
    "IF": "Investee Fund",
    "GPCS": "GP's Capital Subscriptions",
    "RF": "Related Funds",
    "RFM": "Related Firms",
    "PIF": "Primary Investee Fund",
    "DDI": "Discretionary Direct Investment"
}


def replace_acronyms(text, acronym_map):
    doc = nlp(text)  # Process the input text using spaCy

    replaced_text = []
    for token in doc:
        # If the token is an acronym, replace it with its full form from acronym_map
        if token.text in acronym_map:
            # Replace with full form, checking context if needed (you can expand this)
            replaced_text.append(acronym_map[token.text])
        else:
            replaced_text.append(token.text)

    return " ".join(replaced_text)

# Example usage
text = "The CD for the project is approaching, and the GP is discussing the IP extension with the LI."
replaced_text = replace_acronyms(text, acronym_map)
print(replaced_text)

The Closing Date for the project is approaching , and the General Partner is discussing the Investment Period extension with the LI.These all write in Closing Date storage


In [ ]:
#Using regular expression

In [10]:
import re

# Define the acronym mapping
acronym_map = {
    "CD": "Closing Date",
    "IP": "Investment Period",
    "DI": "Direct Investment",
    "AFF": "Affiliate",
    "GP": "General Partner",
    "LI": "Lead Investor",
    "SIF": "Secondary Investee Fund",
    "IF": "Investee Fund",
    "GPCS": "GP's Capital Subscriptions",
    "RF": "Related Funds",
    "RFM": "Related Firms",
    "PIF": "Primary Investee Fund",
    "DDI": "Discretionary Direct Investment"
}

# Function to replace acronyms using regular expressions
def replace_acronyms(text, acronym_map):
    # Replace acronyms in the text
    for acronym, full_form in acronym_map.items():
        regex_pattern = r'\b' + re.escape(acronym) + r'\b'
        text = re.sub(regex_pattern, full_form, text)

    return text

# Example text input
input_text = "The CD is set, and the GP along with LI invested in the IF during the IP. This entire information available in CD memory"

# Replace acronyms in the text
output_text = replace_acronyms(input_text, acronym_map)

print(output_text)


The Closing Date is set, and the General Partner along with Lead Investor invested in the Investee Fund during the Investment Period. This entire information available in Closing Date memory


In [ ]:
#Using spacy and re

In [23]:
import re
import spacy

# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

# Define the acronym mapping
acronym_map = {
    "CD": "Closing Date",
    "IP": "Investment Period",
    "DI": "Direct Investment",
    "AFF": "Affiliate",
    "GP": "General Partner",
    "LI": "Lead Investor",
    "SIF": "Secondary Investee Fund",
    "IF": "Investee Fund",
    "GPCS": "GP's Capital Subscriptions",
    "RF": "Related Funds",
    "RFM": "Related Firms",
    "PIF": "Primary Investee Fund",
    "DDI": "Discretionary Direct Investment"
}

def replace_acronyms(text, acronym_map):
    doc = nlp(text)
    sentences = list(doc.sents)

    # Prepare a mapping of replacements
    replacements = {}

    for sentence in sentences:
        # Extract tokens from the sentence
        tokens = [token.text for token in sentence]

        for acronym, full_form in acronym_map.items():
            # If the acronym is in the sentence, check context
            if acronym in tokens:
                # Collect context words around the acronym
                index = tokens.index(acronym)
                before_context = tokens[max(0, index-5):index]
                after_context = tokens[index+1:index+6]

                # Check if the context suggests the acronym's full form
                before_text = ' '.join(before_context).lower()
                after_text = ' '.join(after_context).lower()

                if any(word in full_form.lower() for word in before_text.split() + after_text.split()):
                    replacements[acronym] = full_form

    # Replace acronyms in the text
    for acronym, full_form in replacements.items():
        text = re.sub(r'\b{}\b'.format(re.escape(acronym)), full_form, text)

    return text



# Example text input
input_text = "The CD is set, and the GP along with LI invested in the IF during the IP. This entire information available in CD memory."


output_text = replace_acronyms(input_text, acronym_map)

print("Output text:")
print(output_text)


Output text:
The Closing Date is set, and the GP along with Lead Investor invested in the Investee Fund during the Investment Period. This entire information available in Closing Date memory.


In [29]:
#Using fuzzy wuzzy

#!pip install fuzzywuzzy python-Levenshtein



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 36.9 MB/s eta 0:00:00


In [33]:
import spacy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Load SpaCy model
nlp = spacy.load("en_core_web_sm")

# Acronym mapping
acronym_map = {
    "CD": "Closing Date",
    "IP": "Investment Period",
    "DI": "Direct Investment",
    "AFF": "Affiliate",
    "GP": "General Partner",
    "LI": "Lead Investor",
    "SIF": "Secondary Investee Fund",
    "IF": "Investee Fund",
    "GPCS": "GP's Capital Subscriptions",
    "RF": "Related Funds",
    "RFM": "Related Firms",
    "PIF": "Primary Investee Fund",
    "DDI": "Discretionary Direct Investment"
}

def replace_acronyms(text):
    doc = nlp(text)
    tokens = [token.text for token in doc]

    # Iterate over tokens and match with acronyms
    new_tokens = []
    for token in tokens:
        # Find best match from acronym_map
        match, score = process.extractOne(token, acronym_map.keys(), scorer=fuzz.partial_ratio)
        if score > 80:  # Adjust threshold as needed
            # Replace with the full term if there's a good match
            new_tokens.append(acronym_map[match])
        else:
            new_tokens.append(token)

    return ' '.join(new_tokens)

# Example usage
text = "The CD for the DI is set for next week, and the GP needs to review the RF and DI."
print(replace_acronyms(text))


The Closing Date for the Direct Investment is set for next week , and the General Partner needs to review the Related Funds and Direct Investment .


In [ ]:
#Using BERT
#pip install transformers torch


In [46]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# Define the acronym mapping
acronym_map = {
    "CD": "Closing Date",
    "IP": "Investment Period",
    "DI": "Direct Investment",
    "AFF": "Affiliate",
    "GP": "General Partner",
    "LI": "Lead Investor",
    "SIF": "Secondary Investee Fund",
    "IF": "Investee Fund",
    "GPCS": "GP's Capital Subscriptions",
    "RF": "Related Funds",
    "RFM": "Related Firms",
    "PIF": "Primary Investee Fund",
    "DDI": "Discretionary Direct Investment"
}


# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

def replace_acronyms_bert(text, acronym_map):
    words = text.split()
    words = [word.strip(".").strip(",") for word in words]
    masked_words = [f"[MASK]" if word in acronym_map else word for word in words]

    # Generate predictions for masked words
    inputs = tokenizer(" ".join(masked_words), return_tensors='pt')
    with torch.no_grad():
        outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)
    predicted_tokens = tokenizer.convert_ids_to_tokens(predictions[0])

    # Replace acronyms with predicted tokens
    replaced_text = " ".join([acronym_map.get(word, word) for word in words])

    return replaced_text

# Example usage
text = "The CD for the DI is set for next week, and the GP needs to review the RF and DDI"
print(replace_acronyms_bert(text, acronym_map))


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The Closing Date for the Direct Investment is set for next week and the General Partner needs to review the Related Funds and Discretionary Direct Investment


In [ ]:
#Using bert

In [52]:
import re
import torch
from transformers import BertTokenizer, BertForTokenClassification
from transformers import pipeline

# Define your acronym map
acronym_map = {
    "CD": "Closing Date",
    "IP": "Investment Period",
    "DI": "Direct Investment",
    "AFF": "Affiliate",
    "GP": "General Partner",
    "LI": "Lead Investor",
    "SIF": "Secondary Investee Fund",
    "IF": "Investee Fund",
    "GPCS": "GP's Capital Subscriptions",
    "RF": "Related Funds",
    "RFM": "Related Firms",
    "PIF": "Primary Investee Fund",
    "DDI": "Discretionary Direct Investment"
}

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForTokenClassification.from_pretrained('bert-base-uncased')

# Define a pipeline for Named Entity Recognition (NER) using BERT
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

def replace_acronyms(text):
    # Tokenize and get predictions
    tokens = tokenizer.tokenize(text)
    inputs = tokenizer.encode(text, return_tensors="pt")

    with torch.no_grad():
        outputs = model(inputs).logits

    # Get the predicted token classes
    predictions = torch.argmax(outputs, dim=2).squeeze().tolist()

    # Create a dictionary of token to prediction
    token_predictions = dict(zip(tokens, predictions))

    # Replace acronyms based on context
    def replace(match):
        acronym = match.group(0)
        if acronym in acronym_map:
            return acronym_map[acronym]
        return acronym

    # Use regex to find acronyms in the text
    acronym_pattern = re.compile(r'\b(?:' + '|'.join(re.escape(key) for key in acronym_map.keys()) + r')\b')
    result_text = acronym_pattern.sub(replace, text)

    return result_text

# Example usage
input_text = "The GP and LI discussed the CD during the IP phase."
input_text = "The CD for the DI is set for next week, and the GP needs to review the RF and DDI."
result = replace_acronyms(input_text)
print(result)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


The Closing Date for the Direct Investment is set for next week, and the General Partner needs to review the Related Funds and Discretionary Direct Investment.These all information stored in Closing Date memory


In [ ]:
##Without acronym map

In [60]:
import spacy
import re

# Load spaCy model
nlp = spacy.load("en_core_web_sm")

def detect_acronyms(text):
    doc = nlp(text)
    acronyms = set()

    for token in doc:
        # Simple pattern to detect uppercase acronyms
        if token.text.isupper() and len(token.text) > 1:
            acronyms.add(token.text)

    return acronyms

# Example usage
input_text = "The GP and LI met to discuss the CD for the IP."
acronyms = detect_acronyms(input_text)
print("Identified Acronyms: ", acronyms)


Identified Acronyms:  {'LI', 'IP', 'CD', 'GP'}


In [61]:
from transformers import pipeline, BertTokenizer, BertForMaskedLM
import torch

# Initialize the tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

def replace_acronyms_with_context(text):
    acronyms = detect_acronyms(text)

    for acronym in acronyms:
        # Create a masked version of the text
        masked_text = text.replace(acronym, '[MASK]')

        # Encode the text
        input_ids = tokenizer.encode(masked_text, return_tensors='pt')
        mask_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1]

        # Predict the masked token
        with torch.no_grad():
            output = model(input_ids)
            predictions = output.logits[0, mask_token_index].topk(1).indices.squeeze().tolist()

        predicted_token = tokenizer.decode(predictions)

        # Replace acronym with predicted expansion
        text = text.replace(acronym, predicted_token)

    return text

# Example usage
input_text = "The GP and LI met to discuss the CD for the IP."
replaced_text = replace_acronyms_with_context(input_text)
print("Original Text: ", input_text)
print("Replaced Text: ", replaced_text)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Original Text:  The GP and LI met to discuss the CD for the IP.
Replaced Text:  The i and i p met to discuss the d e s i g n for the e v e n t.


In [ ]:
from transformers import pipeline

# Initialize the text generation pipeline
generator = pipeline('text-generation', model='EleutherAI/gpt-j-6B')

def generate_acronym_expansion(text, acronym):
    prompt = f"In the context of the following text: '{text}', what does the acronym '{acronym}' likely stand for?"
    result = generator(prompt, max_length=50, num_return_sequences=1)
    return result[0]['generated_text']

def replace_acronyms_with_context(text):
    acronyms = detect_acronyms(text)
    replaced_text = text

    for acronym in acronyms:
        expansion = generate_acronym_expansion(text, acronym)
        # Extract only the relevant part of the generated text
        expansion = expansion.split(":")[-1].strip()
        replaced_text = replaced_text.replace(acronym, expansion)

    return replaced_text

# Example usage
input_text = "The GP and LI met to discuss the CD for the IP."
print("Original Text: ", input_text)
replaced_text = replace_acronyms_with_context(input_text)
print("Replaced Text: ", replaced_text)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from transformers import pipeline

# Initialize the text generation pipeline
generator = pipeline('text-generation', model='EleutherAI/gpt-neo-2.7B')

def infer_acronym_definition(text, acronym):
    prompt = f"In the context of the following text: '{text}', what does the acronym '{acronym}' most likely stand for?"
    result = generator(prompt, max_length=50, num_return_sequences=1)
    return result[0]['generated_text']

# Example usage
input_text = "The GP and LI met to discuss the CD for the IP."
acronym = "GP"


definition = infer_acronym_definition(input_text, acronym)
print(f"Definition for '{acronym}': {definition}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
